In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import zscore
from pylab import *
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import svm
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_58cf7729f092473dbe0e6f96d85b0f8e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='X1fAlcbHpaUWDd_Dlb91OVc-8Hle9F6hrjNuzhzRbQnN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_58cf7729f092473dbe0e6f96d85b0f8e.get_object(Bucket='newproject-donotdelete-pr-hw6kdoeu7cubtm',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_new = pd.read_csv(body)
df_new.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [3]:
df_new['year'] = pd.DatetimeIndex(df_new['date']).year
df_new['month'] = pd.DatetimeIndex(df_new['date']).month
df_new['day_of_year'] =pd.DatetimeIndex(df_new['date']).dayofyear
df_new['week_of_year'] =pd.DatetimeIndex(df_new['date']).weekofyear
df_new['day_of_week'] =pd.DatetimeIndex(df_new['date']).dayofweek

In [4]:
df_new.drop('date',axis=1,inplace=True)

In [5]:
df=df_new[df_new['year']==2017]

In [6]:
x=df.drop('sales',axis=1)

In [7]:
y=df.pop('sales')

In [8]:
x.shape

(182500, 7)

In [9]:
y.shape

(182500,)

In [10]:
x_train,x_test, y_train, y_test=train_test_split(x,y,test_size=0.30, random_state=0)

In [11]:
rfcl=RandomForestRegressor(n_estimators=40, criterion ='mse', max_depth = 7, random_state=1)

In [12]:
rfcl.fit(x_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=7,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
           oob_score=False, random_state=1, verbose=0, warm_start=False)

In [13]:
y_pred = rfcl.predict(x_test)

In [14]:
y_pred

array([98.11148822, 64.20232533, 48.90496884, ..., 64.20232533,
       50.72332255, 56.80309127])

In [15]:
print("The accuracy score for training set is %12.16f" %rfcl.score(x_train,y_train))
print("The accuracy score for testing set is %12.16f" %rfcl.score(x_test,y_test))

The accuracy score for training set is 0.4630174905495261
The accuracy score for testing set is 0.4576317070492001


In [16]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-07-15 10:47:38,881 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [17]:
wml_credentials={
  "apikey": "5uxNE5o3YY5qOOml4Q1Rnhs1BkX8rVvpvNW48XE1SFDm",
  "iam_apikey_description": "Auto-generated for key 234009dc-f398-4479-b7a6-7f077f7f10d4",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/c6ef39a63dd54304b13a28156978aa13::serviceid:ServiceId-1047cd98-a1ef-4b0e-945d-073347f47ba6",
  "instance_id": "1c8f693a-62ac-4e49-91e4-38efc39dff1d",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [18]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [19]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME : "Sneha",
    client.repository.ModelMetaNames.AUTHOR_EMAIL : "uec18445@rmd.ac.in",
    client.repository.ModelMetaNames.NAME : "Goods Count"
}

In [20]:
model_artifact = client.repository.store_model(rfcl,meta_props = model_props)

In [21]:
model_artifact

{'metadata': {'guid': '8d06b249-bc92-4d47-9dec-72cf178eb275',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/published_models/8d06b249-bc92-4d47-9dec-72cf178eb275',
  'created_at': '2020-07-15T10:47:39.251Z',
  'modified_at': '2020-07-15T10:47:39.304Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/published_models/8d06b249-bc92-4d47-9dec-72cf178eb275/learning_configuration',
  'author': {'name': 'Sneha'},
  'name': 'Goods Count',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/published_models/8d06b249-bc92-4d47-9dec-72cf178eb275/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/published_models/8d06b249-bc92-4d47-9dec-72cf178eb275/feedback',
  'latest_version': {'url': 'https:

In [22]:
guid = client.repository.get_model_uid(model_artifact)

In [23]:
guid

'8d06b249-bc92-4d47-9dec-72cf178eb275'

In [24]:
deploy = client.deployments.create(guid, name="Goods Prediction")



#######################################################################################

Synchronous deployment creation for uid: '8d06b249-bc92-4d47-9dec-72cf178eb275' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f'
------------------------------------------------------------------------------------------------




In [25]:
deploy

{'metadata': {'guid': '4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/deployments/4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f',
  'created_at': '2020-07-15T10:47:41.971Z',
  'modified_at': '2020-07-15T10:47:42.625Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Goods Prediction',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/deployments/4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f/online',
  'deployable_asset': {'name': 'Goods Count',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/published_models/8d06b249-bc92-4d47-9dec-72cf178eb275',
   'guid': '8d06b249-bc92-4d47-9dec-72cf178eb275',
   'created_at': '2020-07-15T10:47:41.942Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',
  'status': 'D

In [26]:
scoring_url = client.deployments.get_scoring_url(deploy)

In [27]:
scoring_url

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/1c8f693a-62ac-4e49-91e4-38efc39dff1d/deployments/4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f/online'

In [28]:
client.deployments.list()

------------------------------------  ----------------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME              TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
4789e0b6-e6ee-4b10-94f2-0b917a5a4b8f  Goods Prediction  online  DEPLOY_SUCCESS  2020-07-15T10:47:41.971Z  scikit-learn-0.20  model
05c97403-156e-45e6-b4ed-21279433c6b0  Goods Prediction  online  DEPLOY_SUCCESS  2020-07-15T08:21:29.936Z  scikit-learn-0.20  model
a3453627-438d-43f4-8ea7-216e8db6dec7  Goods Prediction  online  DEPLOY_SUCCESS  2020-07-14T20:27:35.609Z  scikit-learn-0.20  model
8b2528a9-a6a0-480e-81c1-d16e4c266680  Goods Prediction  online  DEPLOY_SUCCESS  2020-07-14T16:24:30.227Z  scikit-learn-0.20  model
------------------------------------  ----------------  ------  --------------  ------------------------  -----------------  -------------
